# Semantic Chunking for Document Processing


# Package Installation and Imports
|


In [1]:
!pip install -q langchain-experimental python-dotenv langchain langchain-text-splitters langchain_community sentence-transformers "chonkie[semantic]" faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
# Clone the repository to access helper functions and evaluation modules
!git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')
# If you need to run with the latest data
# !cp -r RAG_TECHNIQUES/data .

Cloning into 'RAG_TECHNIQUES'...
remote: Enumerating objects: 1769, done.
remote: Counting objects: 100% (1105/1105), done.
remote: Compressing objects: 100% (417/417), done.
remote: Total 1769 (delta 735), reused 690 (delta 688), pack-reused 664 (from 4)
Receiving objects: 100% (1769/1769), 36.51 MiB | 37.05 MiB/s, done.
Resolving deltas: 100% (1121/1121), done.


In [3]:
import os
import sys
import types
from dotenv import load_dotenv


### Define file path

In [4]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)


# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


--2026-01-26 18:03:39--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206372 (202K) [application/octet-stream]
Saving to: ‘data/Understanding_Climate_Change.pdf’

data/Understanding_ 100%[===================>] 201.54K  --.-KB/s    in 0.004s  

2026-01-26 18:03:39 (53.0 MB/s) - ‘data/Understanding_Climate_Change.pdf’ saved [206372/206372]

--2026-01-26 18:03:39--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185

In [4]:
path = "data/Understanding_Climate_Change.pdf"

### Read PDF to string

In [50]:
!pip uninstall -q PyMuPDF -y
!pip install -q PyMuPDF fitz tools

In [5]:
import fitz

def read_pdf_to_string(path):
    """
    Read a PDF document from the specified path and return its content as a string.

    Args:
        path (str): The file path to the PDF document.

    Returns:
        str: The concatenated text content of all pages in the PDF document.

    The function uses the 'fitz' library (PyMuPDF) to open the PDF document, iterate over each page,
    extract the text content from each page, and append it to a single string.
    """
    # Open the PDF document located at the specified path
    doc = fitz.open(path)
    content = ""
    # Iterate over each page in the document
    for page_num in range(len(doc)):
        # Get the current page
        page = doc[page_num]
        # Extract the text content from the current page and append it to the content string
        content += page.get_text()
    return content

content = read_pdf_to_string(path)

In [6]:
print(content)

Understanding Climate Change 
Chapter 1: Introduction to Climate Change 
Climate change refers to significant, long-term changes in the global climate. The term 
"global climate" encompasses the planet's overall weather patterns, including temperature, 
precipitation, and wind patterns, over an extended period. Over the past century, human 
activities, particularly the burning of fossil fuels and deforestation, have significantly 
contributed to climate change. 
Historical Context 
The Earth's climate has changed throughout history. Over the past 650,000 years, there have 
been seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about 
11,700 years ago marking the beginning of the modern climate era and human civilization. 
Most of these climate changes are attributed to very small variations in Earth's orbit that 
change the amount of solar energy our planet receives. During the Holocene epoch, which 
began at the end of the last ice age, human societi

### Split original text to semantic chunks

In [45]:
from chonkie import SemanticChunker
text_splitter = SemanticChunker(embedding_model='sentence-transformers/all-MiniLM-L6-v2', threshold=0.7, chunk_size=512)

In [46]:
docs = text_splitter.chunk(content)

In [47]:
print(docs[0])

Understanding Climate Change 
Chapter 1: Introduction to Climate Change 
Climate change refers to significant, long-term changes in the global climate. The term 
"global climate" encompasses the planet's overall weather patterns, including temperature, 
precipitation, and wind patterns, over an extended period. 


### Create vector store and retriever

In [48]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = embed_model.encode(docs)

embeddings = np.array(embeddings).astype('float32')
dimension = embeddings.shape[1]
# print(dimension)

index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

### Test the retriever

In [49]:
test_query = "What is the main cause of climate change?"

query_vector = embed_model.encode([test_query])
query_vector = np.array(query_vector).astype('float32')

k = 3
distances, indices = index.search(query_vector, k)

# print(indices)
context = [docs[idx] for idx in indices[0]]

output = "\n".join(
    f"{i}. {chunk}\n" + "-" * 80
    for i, chunk in enumerate(context, start=1)
)

print(output)

1. change the amount of solar energy our planet receives. During the Holocene epoch, which 
began at the end of the last ice age, human societies flourished, but the industrial era has seen 

--------------------------------------------------------------------------------
2. children, and outdoor workers, are particularly at risk. Heat mitigation strategies, such as 
cooling centers and public health campaigns, are essential. 
--------------------------------------------------------------------------------
3. champions strengthens the movement. 
Empowerment and Agency 
Empowering individuals and communities to take action fosters a sense of agency and 

--------------------------------------------------------------------------------


![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--semantic-chunking)